# Notkun gervigreindar fyrir teikningu þrívíddarmynda

Nathan HK

In [10]:
import numpy as np
import os
import pywikibot
import sklearn as sk
from sklearn.model_selection import train_test_split
from stl import mesh
import time
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
torch.set_default_device(device)
mappa = '/Users/002-nathan/Desktop/Envalys/envalys-nathan/'  # Change this

## Inngangsorð
Við ætlum að þjálfa gervigreindarlíkan til að teikna þrívíddarmyndir. Hvernig gerum við það?
- Fyrst reyndi ég að nota þrívíddardíla eða "þríla" (e. *voxels*). Ég taldi punktana í hverjum þríl, og þjálfaði líkan með tölunum, eins og líkan sem greinir ljósmyndir. GitHub: https://github.com/nholmesking/envalys-nathan/blob/4fd89a1515c1c3623a5b8cdf8c766858c465e451/Nathan%20HK.ipynb
  - Vandamál #1: Mac-GPU getur ekki neitt með þrívíddargögnum. Ég þurfti að nota CPU, sem er of hægt.
  - Vandamál #2: Hraðinn er $O(n^3)$.
  - Vandamál #3: Það var engin klár leið til að fara frá greiningu yfir á teikningu.
- Þá reyndi ég að nota líkanið 3DShape2VecSet, sem tekur lista yfir punkta og teinkir myndir sem eru eins og listinn.

Tilvísanir:
- https://arxiv.org/pdf/2301.11445
- https://github.com/1zb/3DShape2VecSet

Ég gerði litlar breytingar á upprunaforritið.

## Gögn
Gögnin frá 3DShape2VecSet eru 650 GB samtals; þetta er of mikið fyrir tölvuna mína.

Þessi gögn eru STL-skrár frá Wikimedia Commons. Það eru fimm flokkar:
- líkamshlutar
- byggingar
- rúmfræði
- geimfarartæki
- styttur

### Sækja gögn

In [11]:
flokkar = ['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
skrar = {}
catnum = {}

In [12]:
commons = pywikibot.Site('commons', 'commons')
cn = 0
for a in flokkar:
    print(a)
    cat = pywikibot.Category(commons, 'STL files of ' + a)
    catnum[a] = cn
    cn += 1
    n = 0
    for p in cat.members(member_type=['file']):
        if n % 10 == 0:
            print(n)
        mynd = pywikibot.FilePage(p)
        try:
            tempf = open(mappa + 'STLdata/' + a + '_' + p.title()[5:], 'r')
            tempf.close()
        except FileNotFoundError:
            mynd.download(filename=mappa + 'STLdata/' + a + '_' + p.title()[5:])
        try:
            skrar[a].append(p.title()[5:])
        except KeyError:
            skrar[a] = [p.title()[5:]]
        n += 1
        if n >= 100:
            break

body parts
0
10
20
30
40
50
60
70
80
buildings
0
10
20
geometric shapes
0
10
20
30
40
objects in space
0
10
20
30
40
50
sculptures
0
10
20
30
40
50


### Setja upp gögn fyrir notkun
Við eigum að breyta gögnunum úr STL-sniði í sniðið sem 3DShape2VecSet notar. Þetta snið er með lista yfir punkta eftir hvort þeir séu innan á forminu eða utan, en gögnin frá Wikimedia Commons er ekki með það.

Með ```isInside()``` getum útreiknað hvort lína og þríhyrningur mætast, og í hvora átt. Við búum til tvær línur sem fara beint í Z-áttina frá punktnum (eina línu upp og eina línu niður), og teljum þríhyrningana sem línan mætir; ef talan er slétt, þá er punkturinn utan á forminu, og ef talan er oddatala, þá er punkturinn innan. En það tekur of langan tíma; sumar myndir eru með fleiri en milljón þríhyrninga. Þess vegna ritaði ég tvö reiknirit til að gera þetta fljótara:
- Við deilum myndinni í 32x32x32 þríla og merkjum þá sem eru með formspunkta. Fyrir hvern listapunkt: ef þríllinn punktsins er merktur sem formspunktaþríll, þá getum við ekki neitt, en ef hann er ómerktur, þá útreiknum við hvort listapunkturinn sé innan eða utan, og fyrir alla aðra listapunkta sem eru í þessum þríl, við vitum sjálfkrafa. Og við merkjum alla þríla sem eru beint í allar áttir, fram til myndarendarins eða formspunktaþríls, sem innan/utan.
- Við deilum myndinni í 8x8 ílát eftir X- og Y-hnitum (því við leitum að þríhyrningum í Z-áttina). Í byrjun setjum við alla þríhyrninga í þeirra ílát; sumir þríhyrningar, sem eru á línunni milli íláta, eru settir í tvö eða fleiri ílát. Fyrir hvern listapunkt eigum við bara að leita í rétta ílátinu.

Með þessum tveim reikniritum er sniðbreytingin **40** sinnum fljótari. Ég skrifaði sjálfur allt hérna nema ```isInside()```; sjáðu kóðann fyrir tilvísun.

In [71]:
def isInside(a, randpoint):
    if (a[0] > randpoint[0] and a[3] > randpoint[0] and a[6] > randpoint[0]):
        return 0
    if (a[0] < randpoint[0] and a[3] < randpoint[0] and a[6] < randpoint[0]):
        return 0
    if (a[1] > randpoint[1] and a[4] > randpoint[1] and a[7] > randpoint[1]):
        return 0
    if (a[1] < randpoint[1] and a[4] < randpoint[1] and a[7] < randpoint[1]):
        return 0
 
    # This part is from https://diegoinacio.github.io/creative-coding-notebooks-page/pages/ray-intersection_triangle.html
    e = np.array([0, 0, 1])     # Ray direction
    AB = a[3:6] - a[0:3]               # Oriented segment A to B
    AC = a[6:9] - a[0:3]               # Oriented segment A to C
    n = np.cross(AB, AC)     # Normal vector
    n_ = n/np.linalg.norm(n) # Normalized normal
    if not np.any(np.dot(n_, e)):
        return 0
    d = - np.dot(n_, a[0:3])
    # Finding parameter t
    t = - (np.dot(n_, randpoint) + d)/np.dot(n_, e)
    P = randpoint + t*e
    # Get the resulting vector for each vertex
    # following the construction order
    Pa = np.dot(np.cross(a[3:6] - a[0:3], P - a[0:3]), n_)
    Pb = np.dot(np.cross(a[6:9] - a[3:6], P - a[3:6]), n_)
    Pc = np.dot(np.cross(a[0:3] - a[6:9], P - a[6:9]), n_)

    if t > 0 and (Pa > 0 and Pb > 0 and Pc > 0):
        return 1
        
    e = np.array([0, 0, -1])     # Ray direction
    # Finding parameter t
    t = - (np.dot(n_, randpoint) + d)/np.dot(n_, e)
    P = randpoint + t*e
    # Get the resulting vector for each vertex
    # following the construction order
    Pa = np.dot(np.cross(a[3:6] - a[0:3], P - a[0:3]), n_)
    Pb = np.dot(np.cross(a[6:9] - a[3:6], P - a[3:6]), n_)
    Pc = np.dot(np.cross(a[0:3] - a[6:9], P - a[6:9]), n_)

    if t > 0 and (Pa > 0 and Pb > 0 and Pc > 0):
        return -1
    return 0

In [73]:
for cat in skrar:
    tts = []
    print(cat)
    byrjun = time.time()
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '"')
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/train"')
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/val"')
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_watertight/' + cat + '"')
    for fi in skrar[cat]:
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        utskra_mesh = open(mappa + '3DShape2VecSet/data/ShapeNetV2_watertight/' + cat + '/' + fi + '.npz', 'wb')
        np.savez(utskra_mesh, points=np.array(gogn.v0), scale=1)
        tts.append(fi)
    train, val = train_test_split(tts)
    trainlst = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/train.lst', 'w')
    print('train')
    for fi in train:
        print(fi)
        trainlst.write(fi + '.npz\n')
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        print(gogn.points.shape[0])
        mm = ((min([a[0] for a in gogn.v0]), max([a[0] for a in gogn.v0])),
              (min([a[1] for a in gogn.v0]), max([a[1] for a in gogn.v0])),
              (min([a[2] for a in gogn.v0]), max([a[2] for a in gogn.v0])))
        bucket_id_X = [mm[0][0] + (mm[0][1] - mm[0][0]) * i / 8 for i in range(1, 9)]
        bucket_id_Y = [mm[1][0] + (mm[1][1] - mm[1][0]) * i / 8 for i in range(1, 9)]
        buckets = []
        for i in range(8):
            buckets.append([])
            for j in range(8):
                buckets[i].append([])
        utskra = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/train/' + fi + '.npz', 'wb')
        vol_points = []
        vol_label = []
        scale = (min([mm[0][0], mm[1][0], mm[2][0]]), max([mm[0][1], mm[1][1], mm[2][1]]))
        rescale = lambda x: int((x - scale[0]) * 32 / (scale[1] - scale[0]))
        voxels1 = np.zeros((32, 32, 32), dtype=np.int8)
        voxels2 = np.zeros((32, 32, 32), dtype=np.int8)
        for a in gogn.points:
            p = [0, 0]
            while p[0] < 7 and bucket_id_X[p[0]] < min([a[0], a[3], a[6]]):
                p[0] += 1
                p[1] += 1
            while p[1] < 7 and bucket_id_X[p[1]] < max([a[0], a[3], a[6]]):
                p[1] += 1
            q = [0, 0]
            while q[0] < 7 and bucket_id_Y[q[0]] < min([a[1], a[4], a[7]]):
                q[0] += 1
                q[1] += 1
            while q[1] < 7 and bucket_id_Y[q[1]] < max([a[1], a[4], a[7]]):
                q[1] += 1
            for i in range(p[0], p[1] + 1):
                for j in range(q[0], q[1] + 1):
                    buckets[i][j].append(a)
            for i1 in range(min([rescale(a[0]), rescale(a[3]), rescale(a[6])]), max([rescale(a[0]), rescale(a[3]), rescale(a[6])]) + 1):
                if i1 < 0:
                    i = 0
                elif i1 >= voxels1.shape[0]:
                    i = voxels1.shape[0] - 1
                else:
                    i = i1
                for i2 in range(min([rescale(a[1]), rescale(a[4]), rescale(a[7])]), max([rescale(a[1]), rescale(a[4]), rescale(a[7])]) + 1):
                    if i2 < 0:
                        j = 0
                    elif i2 >= voxels1.shape[0]:
                        j = voxels1.shape[0] - 1
                    else:
                        j = i2
                    for i3 in range(min([rescale(a[2]), rescale(a[5]), rescale(a[8])]), max([rescale(a[2]), rescale(a[5]), rescale(a[8])]) + 1):
                        if i3 < 0:
                            k = 0
                        elif i3 >= voxels1.shape[0]:
                            k = voxels1.shape[0] - 1
                        else:
                            k = i3
                        voxels1[i][j][k] = 1
            n += 1
        for i in range(4096):
            if i % 1000 == 0:
                print('train vol', i, time.time() - byrjun)
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            found = False
            if rescale(randpoint[0]) < 0:
                v0 = 0
            elif rescale(randpoint[0]) >= voxels1.shape[0]:
                v0 = voxels1.shape[0] - 1
            else:
                v0 = rescale(randpoint[0])
            if rescale(randpoint[1]) < 0:
                v1 = 0
            elif rescale(randpoint[1]) >= voxels1.shape[0]:
                v1 = voxels1.shape[0] - 1
            else:
                v1 = rescale(randpoint[1])
            if rescale(randpoint[2]) < 0:
                v2 = 0
            elif rescale(randpoint[2]) >= voxels1.shape[0]:
                v2 = voxels1.shape[0] - 1
            else:
                v2 = rescale(randpoint[2])
            while not found:
                if voxels1[v0][v1][v2] != 0:
                    vox1is1 = True
                    nf[0] += 1
                    break
                if voxels1[v0][v1][v2] == 0 and voxels2[v0][v1][v2] != 0:
                    found = True
                    vol_points.append(randpoint)
                    vol_label.append(max(0, voxels2[v0][v1][v2]))
                    break
                if found:
                    break
                v = [v0, v1, v2]
                while v[0] < voxels1.shape[0] and voxels1[v[0]][v1][v2] == 0:
                    if voxels2[v[0]][v1][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v[0]][v1][v2]))
                        for p in range(v0, v[0]):
                            voxels2[p][v1][v2] = voxels2[v[0]][v1][v2]
                        break
                    v[0] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[0] >= 0 and voxels1[v[0]][v1][v2] == 0:
                    if voxels2[v[0]][v1][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v[0]][v1][v2]))
                        for p in range(v[0] + 1, v0 + 1):
                            voxels2[p][v1][v2] = voxels2[v[0]][v1][v2]
                        break
                    v[0] -= 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[1] < voxels1.shape[1] and voxels1[v0][v[1]][v2] == 0:
                    if voxels2[v0][v[1]][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v[1]][v2]))
                        for p in range(v1, v[1]):
                            voxels2[v0][p][v2] = voxels2[v0][v[1]][v2]
                        break
                    v[1] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[1] >= 0 and voxels1[v0][v[1]][v2] == 0:
                    if voxels2[v0][v[1]][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v[1]][v2]))
                        for p in range(v[1] + 1, v1 + 1):
                            voxels2[v0][p][v2] = voxels2[v0][v[1]][v2]
                        break
                    v[1] -= 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[2] < voxels1.shape[2] and voxels1[v0][v1][v[2]] == 0:
                    if voxels2[v0][v1][v[2]] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v1][v[2]]))
                        for p in range(v2, v[2]):
                            voxels2[v0][v1][p] = voxels2[v0][v1][v[2]]
                        break
                    v[2] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[2] >= 0 and voxels1[v0][v1][v[2]] == 0:
                    if voxels2[v0][v1][v[2]] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v1][v[2]]))
                        for p in range(v[2] + 1, v2 + 1):
                            voxels2[v0][v1][p] = voxels2[v0][v1][v[2]]
                        break
                    v[2] -= 1
                break
            if not found:
                fata = [0, 0]
                while fata[0] < 7 and bucket_id_X[fata[0]] < randpoint[0]:
                    fata[0] += 1
                while fata[1] < 7 and bucket_id_Y[fata[1]] < randpoint[1]:
                    fata[1] += 1
                pct = [0, 0]
                for a in buckets[fata[0]][fata[1]]:
                    isi = isInside(a, randpoint)
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
                if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                    vol_points.append(randpoint)
                    vol_label.append(pct[0] % 2)
                    if pct[0] % 2 == 1:
                        voxels2[v0][v1][v2] = 1
                    else:
                        voxels2[v0][v1][v2] = -1
        near_points = []
        near_label = []
        for i in range(4096):
            if i % 1000 == 0:
                print('train near', i, time.time() - byrjun)
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            fata = [0, 0]
            while fata[0] < 7 and bucket_id_X[fata[0]] < randpoint[0]:
                fata[0] += 1
            while fata[1] < 7 and bucket_id_Y[fata[1]] < randpoint[1]:
                fata[1] += 1
            pct = [0, 0]
            for a in buckets[fata[0]][fata[1]]:
                isi = isInside(a, randpoint)
                if isi == 1:
                    pct[1] += 1
                elif isi == -1:
                    pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                near_points.append(randpoint)
                near_label.append(pct[0] % 2)
        np.savez(utskra,
                 vol_points=np.array(vol_points),
                 vol_label=np.array(vol_label),
                 near_points=np.array(near_points),
                 near_label=np.array(near_label))
    trainlst.close()
    vallst = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/val.lst', 'w')
    print('val')
    for fi in val:
        print(fi)
        vallst.write(fi + '.npz\n')
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        print(gogn.points.shape[0])
        mm = ((min([a[0] for a in gogn.v0]), max([a[0] for a in gogn.v0])),
              (min([a[1] for a in gogn.v0]), max([a[1] for a in gogn.v0])),
              (min([a[2] for a in gogn.v0]), max([a[2] for a in gogn.v0])))
        bucket_id_X = [mm[0][0] + (mm[0][1] - mm[0][0]) * i / 8 for i in range(1, 9)]
        bucket_id_Y = [mm[1][0] + (mm[1][1] - mm[1][0]) * i / 8 for i in range(1, 9)]
        buckets = []
        for i in range(8):
            buckets.append([])
            for j in range(8):
                buckets[i].append([])
        utskra = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/val/' + fi + '.npz', 'wb')
        vol_points = []
        vol_label = []
        scale = (min([mm[0][0], mm[1][0], mm[2][0]]), max([mm[0][1], mm[1][1], mm[2][1]]))
        rescale = lambda x: int((x - scale[0]) * 32 / (scale[1] - scale[0]))
        voxels1 = np.zeros((32, 32, 32), dtype=np.int8)
        voxels2 = np.zeros((32, 32, 32), dtype=np.int8)
        for a in gogn.points:
            p = [0, 0]
            while p[0] < 7 and bucket_id_X[p[0]] < min([a[0], a[3], a[6]]):
                p[0] += 1
                p[1] += 1
            while p[1] < 7 and bucket_id_X[p[1]] < max([a[0], a[3], a[6]]):
                p[1] += 1
            q = [0, 0]
            while q[0] < 7 and bucket_id_Y[q[0]] < min([a[1], a[4], a[7]]):
                q[0] += 1
                q[1] += 1
            while q[1] < 7 and bucket_id_Y[q[1]] < max([a[1], a[4], a[7]]):
                q[1] += 1
            for i in range(p[0], p[1] + 1):
                for j in range(q[0], q[1] + 1):
                    buckets[i][j].append(a)
            for i1 in range(min([rescale(a[0]), rescale(a[3]), rescale(a[6])]), max([rescale(a[0]), rescale(a[3]), rescale(a[6])]) + 1):
                if i1 < 0:
                    i = 0
                elif i1 >= voxels1.shape[0]:
                    i = voxels1.shape[0] - 1
                else:
                    i = i1
                for i2 in range(min([rescale(a[1]), rescale(a[4]), rescale(a[7])]), max([rescale(a[1]), rescale(a[4]), rescale(a[7])]) + 1):
                    if i2 < 0:
                        j = 0
                    elif i2 >= voxels1.shape[0]:
                        j = voxels1.shape[0] - 1
                    else:
                        j = i2
                    for i3 in range(min([rescale(a[2]), rescale(a[5]), rescale(a[8])]), max([rescale(a[2]), rescale(a[5]), rescale(a[8])]) + 1):
                        if i3 < 0:
                            k = 0
                        elif i3 >= voxels1.shape[0]:
                            k = voxels1.shape[0] - 1
                        else:
                            k = i3
                        voxels1[i][j][k] = 1
        for i in range(4096):
            if i % 1000 == 0:
                print('val vol', i, time.time() - byrjun)
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            found = False
            if rescale(randpoint[0]) < 0:
                v0 = 0
            elif rescale(randpoint[0]) >= voxels1.shape[0]:
                v0 = voxels1.shape[0] - 1
            else:
                v0 = rescale(randpoint[0])
            if rescale(randpoint[1]) < 0:
                v1 = 0
            elif rescale(randpoint[1]) >= voxels1.shape[0]:
                v1 = voxels1.shape[0] - 1
            else:
                v1 = rescale(randpoint[1])
            if rescale(randpoint[2]) < 0:
                v2 = 0
            elif rescale(randpoint[2]) >= voxels1.shape[0]:
                v2 = voxels1.shape[0] - 1
            else:
                v2 = rescale(randpoint[2])
            while not found:
                if voxels1[v0][v1][v2] != 0:
                    vox1is1 = True
                    nf[0] += 1
                    break
                if voxels1[v0][v1][v2] == 0 and voxels2[v0][v1][v2] != 0:
                    found = True
                    vol_points.append(randpoint)
                    vol_label.append(max(0, voxels2[v0][v1][v2]))
                    break
                if found:
                    break
                v = [v0, v1, v2]
                while v[0] < voxels1.shape[0] and voxels1[v[0]][v1][v2] == 0:
                    if voxels2[v[0]][v1][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v[0]][v1][v2]))
                        for p in range(v0, v[0]):
                            voxels2[p][v1][v2] = voxels2[v[0]][v1][v2]
                        break
                    v[0] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[0] >= 0 and voxels1[v[0]][v1][v2] == 0:
                    if voxels2[v[0]][v1][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v[0]][v1][v2]))
                        for p in range(v[0] + 1, v0 + 1):
                            voxels2[p][v1][v2] = voxels2[v[0]][v1][v2]
                        break
                    v[0] -= 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[1] < voxels1.shape[1] and voxels1[v0][v[1]][v2] == 0:
                    if voxels2[v0][v[1]][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v[1]][v2]))
                        for p in range(v1, v[1]):
                            voxels2[v0][p][v2] = voxels2[v0][v[1]][v2]
                        break
                    v[1] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[1] >= 0 and voxels1[v0][v[1]][v2] == 0:
                    if voxels2[v0][v[1]][v2] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v[1]][v2]))
                        for p in range(v[1] + 1, v1 + 1):
                            voxels2[v0][p][v2] = voxels2[v0][v[1]][v2]
                        break
                    v[1] -= 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[2] < voxels1.shape[2] and voxels1[v0][v1][v[2]] == 0:
                    if voxels2[v0][v1][v[2]] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v1][v[2]]))
                        for p in range(v2, v[2]):
                            voxels2[v0][v1][p] = voxels2[v0][v1][v[2]]
                        break
                    v[2] += 1
                if found:
                    break
                v = [v0, v1, v2]
                while v[2] >= 0 and voxels1[v0][v1][v[2]] == 0:
                    if voxels2[v0][v1][v[2]] != 0:
                        found = True
                        vol_points.append(randpoint)
                        vol_label.append(max(0, voxels2[v0][v1][v[2]]))
                        for p in range(v[2] + 1, v2 + 1):
                            voxels2[v0][v1][p] = voxels2[v0][v1][v[2]]
                        break
                    v[2] -= 1
                break
            if not found:
                fata = [0, 0]
                while fata[0] < 7 and bucket_id_X[fata[0]] < randpoint[0]:
                    fata[0] += 1
                while fata[1] < 7 and bucket_id_Y[fata[1]] < randpoint[1]:
                    fata[1] += 1
                pct = [0, 0]
                for a in buckets[fata[0]][fata[1]]:
                    isi = isInside(a, randpoint)
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                vol_points.append(randpoint)
                vol_label.append(pct[0] % 2)
        np.savez(utskra,
                 vol_points=np.array(vol_points),
                 vol_label=np.array(vol_label))
    vallst.close()
    print(time.time() - byrjun)
    print('----')

body parts


mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/body parts: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/body parts/train: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/body parts/val: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/body parts: File exists


train
We Are Beautiful – 453703-OSNN-1 – surface.stl
36981
train vol 0 2.978137969970703
train vol 1000 4.929033994674683
train vol 2000 6.959433078765869
train vol 3000 8.899863004684448
train vol 4000 10.916862964630127
train near 0 11.112697839736938
train near 1000 13.026056051254272
train near 2000 15.006965160369873
train near 3000 16.942121982574463
train near 4000 18.84407901763916
BodyParts3D FJ6400 Manubrium.stl
2526
train vol 0 19.10251808166504
train vol 1000 19.854249954223633
train vol 2000 20.602149963378906
train vol 3000 21.302114963531494
train vol 4000 22.079905033111572
train near 0 22.15138006210327
train near 1000 22.883087873458862
train near 2000 23.632456064224243
train near 3000 24.331342935562134
train near 4000 25.04097008705139
Vh-m-kidney-l.stl
83692
train vol 0 28.199594020843506
train vol 1000 32.504307985305786
train vol 2000 36.20829391479492
train vol 3000 39.843138217926025
train vol 4000 43.32884216308594
train near 0 43.7003059387207
train near 100

  n_ = n/np.linalg.norm(n) # Normalized normal



train vol 2000 247.03312492370605
train vol 3000 256.7060601711273
train vol 4000 266.13311791419983
train near 0 267.136727809906
train near 1000 289.23511600494385
train near 2000 310.98649883270264
train near 3000 333.4384949207306
train near 4000 356.53412103652954
We Are Beautiful – 628398-FSAN-1 – solid.stl
235046
train vol 0 365.79760694503784
train vol 1000 370.768620967865
train vol 2000 374.56060910224915
train vol 3000 378.5292418003082
train vol 4000 382.29454016685486
train near 0 382.69039702415466
train near 1000 392.76857805252075
train near 2000 402.43344497680664
train near 3000 412.4369900226593
train near 4000 422.2184729576111
BodyParts3D FJ6272 Xiphoid process.stl
914
train vol 0 423.087051153183
train vol 1000 423.71704506874084
train vol 2000 424.3354790210724
train vol 3000 424.9687077999115
train vol 4000 425.5784010887146
train near 0 425.6337070465088
train near 1000 426.2372989654541
train near 2000 426.8568289279938
train near 3000 427.4965159893036
train 

...

train
Soccer ball.stl
16380
train vol 0 2.1417651176452637
train vol 1000 3.1221349239349365
train vol 2000 3.8463640213012695
train vol 3000 4.548364877700806
train vol 4000 5.279192924499512
train near 0 5.3313939571380615
train near 1000 7.175200939178467
train near 2000 9.039788961410522
train near 3000 10.88428521156311
train near 4000 12.708447933197021
LowPoly Sphere 10.stl
96
train vol 0 12.942200899124146
train vol 1000 13.330533027648926
train vol 2000 13.655332088470459
train vol 3000 13.958165884017944
train vol 4000 14.275717973709106
train near 0 14.30991816520691
train near 1000 14.748002052307129
train near 2000 15.178359031677246
train near 3000 15.618892908096313
train near 4000 16.054049968719482
CP bz 3.stl
97
train vol 0 16.167049169540405
train vol 1000 16.511582136154175
train vol 2000 16.794155836105347
train vol 3000 17.061981201171875
train vol 4000 17.34485697746277
train near 0 17.373193979263306
train near 1000 17.795158863067627
train near 2000 18.21738195

mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/objects in space: File exists


exception (False, 'No lines found, impossible to read')
train
Galileo without probe 3D.stl
16729
train vol 0 2.763597011566162
train vol 1000 3.5715198516845703
train vol 2000 4.175235986709595
train vol 3000 4.791630983352661
train vol 4000 5.32750678062439
train near 0 5.4148828983306885
train near 1000 6.666651725769043
train near 2000 8.161731958389282
train near 3000 9.523839950561523
train near 4000 11.047530889511108
Juno Probe.stl
21514
train vol 0 11.845723152160645
train vol 1000 12.540351867675781
train vol 2000 13.31076693534851
train vol 3000 14.001115083694458
train vol 4000 14.777851104736328
train near 0 14.815653085708618
train near 1000 16.24173092842102
train near 2000 17.395410776138306
train near 3000 18.829210996627808
train near 4000 20.208101987838745
Galileo stowed 3D.stl
16621
train vol 0 20.8456609249115
train vol 1000 21.443946838378906
train vol 2000 22.079621076583862
train vol 3000 22.771026849746704
train vol 4000 23.322992086410522
train near 0 23.36042

mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/sculptures: File exists


train
小山西八朔堰碑.stl
1990726
train vol 0 63.21999979019165
train vol 1000 89.29300665855408
train vol 2000 105.09403276443481
train vol 3000 125.9737138748169
train vol 4000 142.81746768951416
train near 0 144.07260179519653
train near 1000 224.7910258769989
train near 2000 309.0388216972351
train near 3000 390.44868564605713
train near 4000 467.5981297492981
五神名地神塔（横浜市旭区）.stl
1473158
train vol 0 517.2888576984406
train vol 1000 549.36128282547
train vol 2000 581.0574238300323
train vol 3000 611.9173948764801
train vol 4000 642.8432986736298
train near 0 645.8555226325989
train near 1000 716.7149827480316
train near 2000 781.5578486919403
train near 3000 849.477698802948
train near 4000 920.4824998378754
Gisant test.stl
77804
train vol 0 930.2848727703094
train vol 1000 932.5215899944305
train vol 2000 934.8006856441498
train vol 3000 936.9124479293823
train vol 4000 939.2928268909454
train near 0 939.6141548156738
train near 1000 943.2973549365997
train near 2000 946.9750328063965
train 

## Líkan
Líkanið þarf ekki að búa til góðar myndir. Það þarf bara að virka án villna, sem sýnir okkur að við þurfum bara meiri og betri gögn, því vísindagreinin er með góðar myndir.

In [ ]:
os.system('cd ' + mappa)
os.system('torchrun --nproc_per_node=4 3DShape2VecSet/main_ae.py --accum_iter=2 --model ae_d512_m512 '
          '--data_path 3DShape2VecSet/data --output_dir output/ae/ae_d512_m512 --log_dir output/ae/ae_d512_m512 '
          '--num_workers 10 --point_cloud_size 2048 --batch_size 64 --epochs 200 --warmup_epochs 5')

W0824 20:10:57.268526 8579469120 torch/distributed/elastic/multiprocessing/redirects.py:28] NOTE: Redirects are currently not supported in Windows or MacOs.
W0824 20:10:57.279762 8579469120 torch/distributed/run.py:779] 
W0824 20:10:57.279762 8579469120 torch/distributed/run.py:779] *****************************************
W0824 20:10:57.279762 8579469120 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0824 20:10:57.279762 8579469120 torch/distributed/run.py:779] *****************************************


['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
Model = AutoEncoder(
  (cross_attend_blocks): ModuleList(
    (0): PreNorm(
      (fn): Attention(
        (to_q): Linear(in_features=512, out_features=512, bias=False)
        (to_kv): Linear(in_features=512, out_features=1024, bias=False)
        (to_out): Linear(in_features=512, out_features=512, bias=True)
        (drop_path): Identity()


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Traceback (most recent call last):
  File "/Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/main_ae.py", line 256, in

In [ ]:
os.system('python 3DShape2VecSet/eval.py')

In [ ]:
os.system('torchrun --nproc_per_node=4 3DShape2VecSet/main_class_cond.py --accum_iter 2 --model kl_d512_m512_l8_d24_edm '
          '--ae kl_d512_m512_l8 --ae-pth output/ae/kl_d512_m512_l8/checkpoint-199.pth  --output_dir output/dm/kl_d512_m512_l8_d24_edm '
          '--log_dir output/dm/kl_d512_m512_l8_d24_edm --num_workers 64 --point_cloud_size 2048 --batch_size 64 --epochs 1000'
          '--data_path 3DShape2VecSet/data')

Tölvan mín hrundi þegar ég reyndi að þjálfa líkanið. En þjálfunin byrjaði án villna.

# Lokaorð
Hvert förum við héðan? 3DShape2VecSet er með margar takmarkanir.
- Það kann bara að búa til einfaldar myndir, eins og "stóll". Með greiningarlíkani getum við mögulega búið til fjölfaldar myndir. Ef notandi segir "stóll með kodda", þá býr líkanið okkar til mynd af stóli og mynd af kodda, sem líkanið sameinar.
- Það skilur ekki mannamál.
- Það er ekki með mikil gögn. Með mínu forriti hérna getum við búið til eins mikil gögn og við viljum; við getum tekið myndir frá notendum og búum til lista yfir punkta, og notað þá til að þjálfa líkanið.

Ég er viss um að, með meiri gögn og betri tölvu, við getum að minnsta kosti eitt af þessum.